## ボートレースのレース情報をクロールしpickleファイルに保存
- クロール元：ボートレース 公式サイト（https://boatrace.jp/owpc/pc/race/racelist?rno=12&jcd=01&hd=20210325など）
- 保存先：'./crawledData/　以下。日にちごとにファイルを作成し保存

### 使い方
- 上から順番に各種モジュールをロード
- 「実行部分」セルの最初の以下の行にてクロールを行う日付を指定

　　　　　　　　`　hd_list = ["2021/02/0" + str(day) for day in range(1,10)]`
### 注意点
- （多分）beautifulsoupが新しいversionだとtagを取ってくるときのスペースが無視されてエラーになる

In [1]:
from datetime import datetime
from datetime import timedelta
from http.client import RemoteDisconnected
from bs4 import BeautifulSoup
import urllib.request
import time
import pandas as pd
import re
from tqdm.notebook import tqdm

### web pageから情報を取ってきてpandas dfに格納するモジュール

In [2]:
def scrape_racelist(soup, rno, jcd, hd):
    """
    racelistのページに書かれている情報をクロール
    :return:
    """
    table = soup.find(class_="contentsFrame1_inner").find_all(class_="table1")[1]
    rows = table.find_all("tbody", {"class": "is-fs12"})
    
    race_result_dict_list = []
    
    for i, row in enumerate(rows, 1):
        race_result_dict = {"date": "-".join([hd[0:4], hd[5:7], hd[8:10]]),
                            "venue": jcd, "raceNumber": rno[:-1]
                           }
        # 枠
        race_result_dict["枠"] = i
        # racer id
        race_result_dict["racer_id"] = row.find(class_="is-fs11").text.split("\n")[1][-6:-2]
        race_result_dict["racer_class"] = row.find(class_="is-fs11").text.split("\n")[2][-2:]

        # 選手名。最後の[1:-1]は改行を削除するため
        racer_name = row.find(class_="is-fs18 is-fBold").text[1:-1]
        
        # race_result_listの要素としてクロールした結果のリストを追加
        race_result_dict["racer_name"] = racer_name

        # racer data
        racer_column_3 = row.find_all("td", {"class": "is-lineH2"})[0].text.split("\n")
        race_result_dict["num_false_start"] = racer_column_3[1][-3:-1]
        race_result_dict["num_late_start"] = racer_column_3[2][-3:-1]

        # crawl motor data
        motor_column = row.find_all("td", {"class": "is-lineH2"})[3].text.split("\n")
        race_result_dict["motorNo"] = motor_column[1][-4:-1]
        race_result_dict["モーター2連率"] = motor_column[2][-7:-1]
        race_result_dict["モーター3連率"] = motor_column[3][-7:-1]

        # crawl boat data
        boat_column = row.find_all("td", {"class": "is-lineH2"})[4].text.split("\n")
        race_result_dict["boatNo"] = boat_column[1][-4:-1]
        race_result_dict["ボート2連率"] = boat_column[2][-7:-1]
        race_result_dict["ボート3連率"] = boat_column[3][-7:-1]
        
        race_result_dict_list.append(race_result_dict)
        
    # dictをdfに変換
    race_result_df = pd.DataFrame.from_dict(race_result_dict_list)

    time.sleep(0.1)

    return race_result_df

In [3]:
def scrape_beforeinfo(soup, rno, jcd, hd):
    """
    exhibitionの情報など、直前情報ページに書かれている情報をクロール
    :param soup:
    :param rno:
    :param jcd:
    :param hd:
    :return:

    # TODO: プロペラ
    # TODO: 部品交換
    # TODO: 前走成績
    # TODO: 調整重量 (adjustment weight) (kg)
    # TODO: 風向き

    """
    race_result_dict_list = []
    
    table = soup.find(class_="contentsFrame1_inner").find_all(class_="table1")[1]
    rows = table.find_all("tbody", {"class": "is-fs12"})
    
    table2 = soup.find(class_="contentsFrame1_inner").find_all(class_="table1")[2]
    rows2 = table2.find_all("tr")


    for i, (row, row2) in enumerate(zip(rows, rows2[2:]), 1):
        
        race_result_dict = {"date": "-".join([hd[0:4], hd[5:7], hd[8:10]]),
                        "venue": jcd,
                        "raceNumber": rno[:-1]
                        }
        # 枠
        race_result_dict["枠"] = i
        
        # 水面気象情報
        table3 = soup.find(class_="contentsFrame1_inner").find(class_="weather1")
        weather_data = (table3.find_all(class_="weather1_bodyUnitLabelData"))
        weather_string = table3.find_all(class_="weather1_bodyUnitLabelTitle")

        race_result_dict["temperature"] = weather_data[0].text[:-1]
        race_result_dict["weather"] = weather_string[1].text
        race_result_dict["wind_speed"] = weather_data[1].text[:-1]
        race_result_dict["water_temperature"] = weather_data[2].text[:-1]
        race_result_dict["wave_height"] = weather_data[3].text[:-2]

        # racer weight (kg)
        # 書いていないことがあり、その場合エラーになる
        race_result_dict["weight"] = row.find("td", {"rowspan": "2"}).text[:-2]

        # 展示タイム
        race_result_dict["exhibitionTime"] = row.find_all("td", {"rowspan": "4"})[3].text

        # チルト角度
        race_result_dict["tilt"] = row.find_all("td", {"rowspan": "4"})[4].text
        
        # 展示競争での進入コース
        race_result_dict["exhibition_cource"] = row2.find_all("span")[0].text
        # 展示start time (ST, flyng, late)
        ex_st_ = row2.find_all("span")[3].text
        if len(ex_st_) == 3:
            race_result_dict["exhibition_ST"] = ex_st_
            race_result_dict["flying"] = 0
            race_result_dict["late"] = 0

        elif len(ex_st_) == 4:
            race_result_dict["exhibition_ST"] = ex_st_[1:]
            if ex_st_[0] == "F":
                race_result_dict["flying"] = 1
                race_result_dict["late"] = 0
            # elif ex_st_[0] == "L":
            #     race_result_dict["late_{0}".format(i)] = 1
            else:
                raise Exception("{0}号艇ex_stが予定外（{1}）".format(i, ex_st_))
        elif len(ex_st_) == 1:
            if ex_st_[0] == "L":
                race_result_dict["exhibition_ST"] = None
                race_result_dict["late"] = 1
            else:
                raise Exception("{0}号艇ex_stが予定外（{1}）".format(i, ex_st_))

        else:
            raise Exception("{0}号艇ex_stが予定外（{1}）".format(i, ex_st_))
        
        # 最初に定義したリストに辞書型のデータを追加
        race_result_dict_list.append(race_result_dict)

    # dictを入れたlistをdfに変換
    beforeinfo_df = pd.DataFrame.from_dict(race_result_dict_list)

    time.sleep(0.1)

    return beforeinfo_df

In [4]:
def scrape_raceresult(soup, rno, jcd, hd):
    
    race_result_dict_list = []
    
    table = soup.find(class_="contentsFrame1_inner").find_all(class_="table1")[1]
    rows = table.find_all("tbody")
    
    for row in rows:
        race_result_dict = {"date": "-".join([hd[0:4], hd[5:7], hd[8:10]]),
                            "venue": jcd,
                            "raceNumber": rno[:-1]
                            }
        race_result_dict["着順"] = row.find_all("td")[0].text
        # 枠番はintegerにしておかないとconcatした時に別の行として扱われてしまう
        race_result_dict["枠"] = int(row.find_all("td")[1].text)
        race_result_dict["タイム"] = row.find_all("td")[3].text
        
        # 最初に定義したリストに辞書型のデータを追加
        race_result_dict_list.append(race_result_dict)
    
    # dictを入れたlistをdfに変換
    raceresult_df = pd.DataFrame.from_dict(race_result_dict_list)

    time.sleep(0.1)

    return raceresult_df

### そのほかcrawl, scrapeに必要なモジュール

In [5]:
def make_url(crawl_key, rno, jcd, hd):
    """
    :param crawl_key: 何をcrawleするか。選択肢は、"odds3t"（オッズ）, "racelist"(出走表）,
    "beforeinfo" (直前情報）もしくは"raceresult" (レース結果)
    :param rno: レース番号。8Rなど、1-12の数字 + R をstrで
    :param jcd: 会場名。"桐　生"、"びわこ"など
    :param hd: holding day (レース開催日)、2019/03/28などyyyy/mm/ddの形で入力（strで）
    :return dds_url: 公式サイト最終オッズが書かれているページのurl. これを使ってcrawlする
    """
    jcd_dict =  {"桐　生": "01", "戸　田": "02", "江戸川": "03", "平和島": "04", "多摩川": "05", "浜名湖": "06", "蒲　郡": "07", "常　滑": "08",
                "　津　": "09", "三　国": "10", "びわこ": "11", "住之江": "12", "尼　崎": "13", "鳴　門": "14", "丸　亀": "15", "児　島": "16",
                "宮　島": "17", "徳　山": "18", "下　関": "19", "若　松": "20", "芦　屋": "21", "福　岡": "22", "唐　津": "23", "大　村": "24"
                }
    rno = rno[:-1]
    hd = hd[0:4] + hd[5:7] + hd[8:10]

    odds_url = "http://boatrace.jp/owpc/pc/race/" + crawl_key + "?rno=" + rno + "&jcd=" + jcd_dict[jcd] + "&hd=" + hd

    return odds_url


def html_parser(site_url):
    headers = {
        "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:47.0) Gecko/20100101 Firefox/47.0",
    }

    try:
        request = urllib.request.Request(url=site_url, headers=headers)
        response = urllib.request.urlopen(request)

        html = response.read().decode('utf-8')
        soup = BeautifulSoup(html, 'lxml')

    # データベース作成の際、remotedisconnectedになった場合,そのレースをパス
    except RemoteDisconnected:
        print("remote disconnected error !")
        return None

    except ConnectionResetError:
        print("Connection Reset error !")
        return None

    return soup

def get_extractor(crawl_key):
    
    """
    クロール先に応じたcrawlerを用意
    
    """
    
    extractor_dict = {"racelist": scrape_racelist,
                      "beforeinfo": scrape_beforeinfo,
                      "raceresult": scrape_raceresult,
                      }
    
    return extractor_dict[crawl_key]

### 実行部分

In [ ]:
hd_list = ["2021/02/" + str(day) for day in range(23,29)]

crawl_key_list = ["racelist", "beforeinfo", "raceresult"]
jcd_list =  ["桐　生", "戸　田", "江戸川", "平和島", "多摩川", "浜名湖", "蒲　郡", "常　滑",
                "　津　", "三　国", "びわこ", "住之江", "尼　崎", "鳴　門", "丸　亀", "児　島",
                "宮　島", "徳　山", "下　関", "若　松", "芦　屋", "福　岡", "唐　津", "大　村"
            ]

for hd in hd_list:
    print("{0} のデータをクロール中".format(hd))

    # 1日単位でデータを集めてファイルに保存する
    today_race_df_list = []

    for jcd in tqdm(jcd_list):
        for i in range(1, 13):
            rno = str(i) + "R"

            # その日レースがない場所は飛ばすためのtry-except         
            try:
                # 色々なkeyに対してクロールして特定のレースの情報がまとまったdfを作る
                race_info_df_list = []

                for crawl_key in crawl_key_list:
                    raceResult_url = make_url(crawl_key, rno, jcd, hd)

                    # パース
                    soup = html_parser(raceResult_url)

                    # extractorの指定
                    the_extractor = get_extractor(crawl_key)

                    # 対象サイトをcrawl
                    race_information_df = the_extractor(soup, rno, jcd, hd)
                    race_information_df = race_information_df.set_index(["date", "venue", "raceNumber", "枠"])

                    race_info_df_list.append(race_information_df)

                this_race_df =pd.concat(race_info_df_list, axis=1)
                # 今回のレースのデータを本日のデータを集めたリストに格納
                today_race_df_list.append(this_race_df)

            except IndexError:
                # print(hd + " " + jcd + rno +"データなし")
                pass

    # 本日のレースデータを集めたリストをdfに変換    
    today_race_df = pd.concat(today_race_df_list, axis = 0)

    # pickleファイルで保存
    today_race_df.to_pickle('./crawledData/{0}.pkl'.format("".join(hd.split("/"))))

2021/02/23 のデータをクロール中


  0%|          | 0/24 [00:00<?, ?it/s]

2021/02/24 のデータをクロール中


  0%|          | 0/24 [00:00<?, ?it/s]

In [ ]:
# ファイル内容確認用
pd.read_pickle('./crawledData/{0}.pkl'.format("".join(hd.split("/"))))

In [ ]:
# crawlerの動きを確認する用

crawl_key = "raceresult"
jcd =  "戸　田"
hd = "2021/03/10"
rno = "1R"

raceResult_url = make_url(crawl_key, rno, jcd, hd)
print(raceResult_url)

# パース
soup = html_parser(raceResult_url)

# extractorの指定
the_extractor = get_extractor(crawl_key)

# 対象サイトをcrawl
race_information_df = the_extractor(soup, rno, jcd, hd)
race_information_df = race_information_df.set_index(["date", "venue", "raceNumber", "枠"])
race_information_df